In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

In [1]:
import os
import re
import random
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, 
)

warnings.filterwarnings('ignore')

In [2]:
MODEL_CHECKPOINT = 'training_results/testrun/ainize_kobart_news_testrun/checkpoint-507'
DATA_PATH = 'data/original/train.csv'

In [3]:
data_df = pd.read_csv(DATA_PATH)

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [5]:
idx = random.randint(0, len(data_df)-1)

sentence = data_df.context.iloc[idx]

inputs = tokenizer(sentence, max_length=1026, truncation=True, return_tensors="pt")
inputs = {k: v.cpu() for k, v in inputs.items() if k != 'token_type_ids'}
print(inputs['input_ids'].shape)

logits = model(**inputs).logits
print(logits.shape)

keys = logits.argmax(-1).squeeze().tolist()

tokenizer.decode(keys)

torch.Size([1, 181])
torch.Size([1, 181, 30000])


'<s>제2정 제4항 예산 예산결산특별위원회 구성결의안을 상정함</s>회 세입,세출 추가경정예산안 심의에 따른 예산결산특별위원회 구성은 의원님들이 사전에 양해하여 주신대로 특별위원회를 구성하도록 의견이습니다.</s>결산특별위원회 위원은 일곱 분의 의원님으로 구성하도록 의견이 모아진 사항으로 제가 특별위원회 위원을 지명하겠습니다.</s>결산특별위원회 위원으로는 반광홍 부의장님, 이한철 의원님, 윤병승 의원님, 박희남 의원님, 강연수 의원님, 이준구 의원님, 정지태 의원님으로 구성하고자 함 그리고 특별위원회 운영기간은 10월 4일부터 10월 6일까지 3일간으로 하고자 함</s> 제의한 예산결산특별위원회 구성안에 대하여 의원들! 이의 없으십니까? (「없습니다」하는 의원 있음) 이의가 없으므로 가결되었음을 선포함</s>'

# Generate

In [ ]:
model.cuda()

In [ ]:
sample = data_df.context.iloc[1]

In [ ]:
max_input_length = 1026

input_ids, _, _ = tokenizer(sample, max_length=max_input_length, truncation=True, return_tensors="pt").values()
input_ids = input_ids.cuda()

## Greedy Search

In [ ]:
outputs = model.generate(input_ids, max_length=512)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

## Beam Search

In [ ]:
outputs = model.generate(
    input_ids, max_length=512, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

## Sampling

In [ ]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=0,
    temperature=0.7,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

### Top-K Sampling

In [ ]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=50,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

### Top-p (nucleus) sampling

In [ ]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_p=0.92, 
    top_k=0,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

#### top_k = 50 and set top_p = 0.95 and num_return_sequences = 3

In [ ]:
outputs = model.generate(
    input_ids, max_length=512, 
    do_sample=True, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=5,
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)